In [13]:
import pyspark
from pyspark.context import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [14]:
spark = SparkSession.builder.appName("Vu dep trai").getOrCreate()

airportsSchema = StructType() \
    .add("id", "integer") \
    .add("airport_name", "string") \
    .add("city", "string") \
    .add("country", "string") \
    .add("iata", "string") \
    .add("icao", "string") \
    .add("latitude", "float") \
    .add("longitude", "float") \
    .add("altitude", "integer") \
    .add("timezone", "byte") \
    .add("dst", "string") \
    .add("tz_database_timezone", "string") \
    .add("type", "string") \
    .add("source", "string")

routesSchema = StructType() \
    .add("airline", "string") \
    .add("airline_id", "integer") \
    .add("source_airport", "string") \
    .add("source_airport_id", "integer") \
    .add("destination_airport", "string") \
    .add("destination_airport_id", "integer") \
    .add("codeshare", "string") \
    .add("stops", "integer") \
    .add("equipment", "string")


In [15]:
airports = spark.read.csv("data/airports.dat", schema=airportsSchema)
airports.createOrReplaceTempView("airports")
airports.show(10)

+---+--------------------+------------+----------------+----+----+---------+----------+--------+--------+---+--------------------+-------+-----------+
| id|        airport_name|        city|         country|iata|icao| latitude| longitude|altitude|timezone|dst|tz_database_timezone|   type|     source|
+---+--------------------+------------+----------------+----+----+---------+----------+--------+--------+---+--------------------+-------+-----------+
|  1|      Goroka Airport|      Goroka|Papua New Guinea| GKA|AYGA| -6.08169|   145.392|    5282|      10|  U|Pacific/Port_Moresby|airport|OurAirports|
|  2|      Madang Airport|      Madang|Papua New Guinea| MAG|AYMD| -5.20708|   145.789|      20|      10|  U|Pacific/Port_Moresby|airport|OurAirports|
|  3|Mount Hagen Kagam...| Mount Hagen|Papua New Guinea| HGU|AYMH| -5.82679|   144.296|    5388|      10|  U|Pacific/Port_Moresby|airport|OurAirports|
|  4|      Nadzab Airport|      Nadzab|Papua New Guinea| LAE|AYNZ|-6.569803| 146.72598|     23

In [16]:
routes = spark.read.csv("data/routes.dat", schema=routesSchema)
routes.show(10)
routes.createOrReplaceTempView("routes")

+-------+----------+--------------+-----------------+-------------------+----------------------+---------+-----+---------+
|airline|airline_id|source_airport|source_airport_id|destination_airport|destination_airport_id|codeshare|stops|equipment|
+-------+----------+--------------+-----------------+-------------------+----------------------+---------+-----+---------+
|     2B|       410|           AER|             2965|                KZN|                  2990|     null|    0|      CR2|
|     2B|       410|           ASF|             2966|                KZN|                  2990|     null|    0|      CR2|
|     2B|       410|           ASF|             2966|                MRV|                  2962|     null|    0|      CR2|
|     2B|       410|           CEK|             2968|                KZN|                  2990|     null|    0|      CR2|
|     2B|       410|           CEK|             2968|                OVB|                  4078|     null|    0|      CR2|
|     2B|       

In [17]:
print("Number of airports: ")
spark.sql("SELECT COUNT(*) FROM airports").show()

Number of airports: 
+--------+
|count(1)|
+--------+
|    7698|
+--------+



In [18]:
print("Number of airports DISTINCT: ")
spark.sql("SELECT COUNT(DISTINCT airport_name) FROM airports").show()

Number of airports DISTINCT: 
+----------------------------+
|count(DISTINCT airport_name)|
+----------------------------+
|                        7658|
+----------------------------+



In [19]:
print("Filtering airports in Greenland")
spark.sql("SELECT * FROM airports WHERE country = 'Greenland'").show()

Filtering airports in Greenland
+----+--------------------+--------------------+---------+----+----+---------+----------+--------+--------+---+--------------------+-------+-----------+
|  id|        airport_name|                city|  country|iata|icao| latitude| longitude|altitude|timezone|dst|tz_database_timezone|   type|     source|
+----+--------------------+--------------------+---------+----+----+---------+----------+--------+--------+---+--------------------+-------+-----------+
|   7|  Narsarsuaq Airport|        Narssarssuaq|Greenland| UAK|BGBW|  61.1605|   -45.426|     112|      -3|  E|     America/Godthab|airport|OurAirports|
|   8|Godthaab / Nuuk A...|            Godthaab|Greenland| GOH|BGGH|  64.1909|  -51.6781|     283|      -3|  E|     America/Godthab|airport|OurAirports|
|   9|Kangerlussuaq Air...|         Sondrestrom|Greenland| SFJ|BGSF| 67.01222|-50.711605|     165|      -3|  E|     America/Godthab|airport|OurAirports|
|  10|      Thule Air Base|               Thule|Gr

In [20]:
print("Joining routes and airports")
joined_tabel = spark.sql("SELECT * FROM routes JOIN airports ON routes.source_airport_id = airports.id")
joined_tabel.createOrReplaceTempView("joined_tabel")
joined_tabel.show(10)

Joining routes and airports
+-------+----------+--------------+-----------------+-------------------+----------------------+---------+-----+---------+----+--------------------+-----------+-------+----+----+--------+---------+--------+--------+---+--------------------+-------+-----------+
|airline|airline_id|source_airport|source_airport_id|destination_airport|destination_airport_id|codeshare|stops|equipment|  id|        airport_name|       city|country|iata|icao|latitude|longitude|altitude|timezone|dst|tz_database_timezone|   type|     source|
+-------+----------+--------------+-----------------+-------------------+----------------------+---------+-----+---------+----+--------------------+-----------+-------+----+----+--------+---------+--------+--------+---+--------------------+-------+-----------+
|     2B|       410|           AER|             2965|                KZN|                  2990|     null|    0|      CR2|2965|Sochi Internation...|      Sochi| Russia| AER|URSS| 43.4499|  

In [21]:
print("Joining routes and airports")
joined_tabel = spark.sql("SELECT * FROM routes JOIN airports ON routes.source_airport_id = airports.id")
joined_tabel.show(10)

Joining routes and airports
+-------+----------+--------------+-----------------+-------------------+----------------------+---------+-----+---------+----+--------------------+-----------+-------+----+----+--------+---------+--------+--------+---+--------------------+-------+-----------+
|airline|airline_id|source_airport|source_airport_id|destination_airport|destination_airport_id|codeshare|stops|equipment|  id|        airport_name|       city|country|iata|icao|latitude|longitude|altitude|timezone|dst|tz_database_timezone|   type|     source|
+-------+----------+--------------+-----------------+-------------------+----------------------+---------+-----+---------+----+--------------------+-----------+-------+----+----+--------+---------+--------+--------+---+--------------------+-------+-----------+
|     2B|       410|           AER|             2965|                KZN|                  2990|     null|    0|      CR2|2965|Sochi Internation...|      Sochi| Russia| AER|URSS| 43.4499|  

In [22]:
print("Count the number of flights arriving in each country")
spark.sql(
    "SELECT country, COUNT(airport_name) AS airport_count "
    "FROM joined_tabel "
    "GROUP BY country"
).show(truncate=False)

Count the number of flights arriving in each country
+--------------+-------------+
|country       |airport_count|
+--------------+-------------+
|Chad          |9            |
|Paraguay      |22           |
|Anguilla      |6            |
|Russia        |1829         |
|Yemen         |93           |
|Senegal       |70           |
|Sweden        |474          |
|Kiribati      |7            |
|Guyana        |9            |
|Jersey        |38           |
|Philippines   |395          |
|Burma         |104          |
|Eritrea       |8            |
|Tonga         |5            |
|Norfolk Island|3            |
|Djibouti      |23           |
|Malaysia      |565          |
|Singapore     |408          |
|Fiji          |69           |
|Turkey        |977          |
+--------------+-------------+
only showing top 20 rows

